In [55]:
# Scraping pitching data from baseball reference
# Jordan McGregor

In [56]:
#list of libraries being imported
from bs4 import BeautifulSoup
import urllib
import collections
import pandas as pd
import numpy as np

In [57]:
#hard-coded for now
currentTeam = "TorontoBlueJays"
year = "2015"
currentTeamURL = "http://www.baseball-reference.com/teams/TOR/2015-schedule-scores.shtml"

In [58]:
#opening the team schedule page 
team_schedule = urllib.urlopen(currentTeamURL).read()
team_schedule = BeautifulSoup(team_schedule)

In [59]:
team_schedule_table = team_schedule.find(id="team_schedule")

In [60]:
#function used to convert the date from the form (year, month_name day_number) to (yyyymmdd)
def dateConversion(oldDate):
    newDate = ""
    month = ""
    day_num = ""
    dateSystemDict = {"Jan": "01",
                      "Feb": "02",
                      "Mar": "03",
                      "Apr": "04",
                      "May": "05",
                      "Jun": "06",
                      "Jul": "07",
                      "Aug": "08",
                      "Sep": "09",
                      "Oct": "10",
                      "Nov": "11",
                      "Dec": "12"}
    oldDate = oldDate.split(' ') #the default is space, but it is used here to be explicit
    splitSize = len(oldDate)
    month = dateSystemDict[oldDate[splitSize-2]]
    day_num = oldDate[splitSize-1]
    if (len(day_num) == 1):
        day_num = "0" + day_num
    elif (len(day_num) < 1 or len(day_num) > 2):
        print "error getting date from schedule table"
    newDate = year + month + day_num
    return newDate


In [61]:
#function to change the team abbreviation if needed
#the team abrs used in the table do not match up with the ones used in the url for boxscores
def teamAbrConversion(teamAbr):
    teamAbrDict = {"NYY":"NYA",
                   "TBR":"TBA",
                   "WSN":"WAS",
                   "NYM":"NYN",
                   "CHW":"CHA",
                   "KCR":"KCA",
                   "LAA":"ANA"}
    if (teamAbrDict.has_key(teamAbr)):
        return teamAbrDict[teamAbr]
    return teamAbr

In [62]:
#function to loop through a team's schedule a grab all the url needed to request boxscores
#hard-coded Toronto as the home team
def getListofOpps(schedule_table):
    listOfOpps = []
    #get a list of all the rows in the schedule table
    team_schedule_rows = schedule_table.find_all('tr')
    team_schedule_rows.pop(0)
    for a_row in team_schedule_rows:
        try:
            columns = a_row.find_all("td")
            opp = columns[6]
            listOfOpps.append(opp)
        except:
            continue
    return listOfOpps

In [63]:
testURL = "http://www.baseball-reference.com/boxes/NYA/NYA201504060.shtml"

In [64]:
boxscore_page = urllib.urlopen(testURL).read()
boxscore_page = BeautifulSoup(boxscore_page)

In [65]:
#function to loop through a team's schedule a grab all the url needed to request boxscores
#hard-coded Toronto as the home team
def getListofBoxscoreURLs(schedule_table):
    listOfUrls = []
    #get a list of all the rows in the schedule table
    team_schedule_rows = schedule_table.find_all('tr')
    team_schedule_rows.pop(0)
    for a_row in team_schedule_rows:
        try:
            columns = a_row.find_all("td")
            base_url = "http://www.baseball-reference.com/boxes/"
            end_url = "0.shtml"
            oldTeamAbr = a_row.find(id="Click for standings on this date")
            date = dateConversion(a_row.find(title="Click for standings on this date").get_text())
            #find the home team by first looking in the row for an @ symbol
            homeTeamAbr = ""
            if ("@" in columns[5].get_text()):
                homeTeamAbr = teamAbrConversion(columns[6].get_text())
            else:
                homeTeamAbr = "TOR" #hard-coded
            url = base_url + homeTeamAbr + '/' + homeTeamAbr + date + end_url
            listOfUrls.append(url.encode('utf-8'))
        except:
            continue
    return listOfUrls
        

In [83]:
def getPitchingStats(page,opp):
    boxscore_batting_table = page.find(id="TorontoBlueJayspitching")
    pitcher_rows = boxscore_batting_table.find_all('tr')
    pitcher_rows.pop(0)
    gamePitchingStats = []
    for pitcher_info in pitcher_rows:
        record = {"name":"",
                  "IP": 0,
                  "H": 0,
                  "BF": 0,
                  "ER": 0,
                  "BB": 0,
                  "SO": 0,
                  "team_faced":""}
        attributes = pitcher_info.find_all('td')
        name = attributes[0].get_text()

        record["name"] = name.strip().encode('utf-8')
        record["IP"] = (attributes[1].get_text()).encode('utf-8')
        record["H"] = (attributes[2].get_text()).encode('utf-8')
        record["BF"] = (attributes[9].get_text()).encode('utf-8')
        record["R"] = (attributes[10].get_text()).encode('utf-8')
        record["team_faced"] = opp
        gamePitchingStats.append(record)
    return gamePitchingStats

In [84]:
#function to get all urls from a team's schedule then get all the batter information
def getAllTeamBattingStats(t_schedule):
    #variable to store all data records to be inputted into excel
    data = []
    #get all the urls
    urlList = getListofBoxscoreURLs(t_schedule)
    oppList = getListofOpps(t_schedule)
    #open each boxscore page to grab info
    for i in range (0,2):
        boxscore_page = urllib.urlopen(urlList[i]).read()
        boxscore_page = BeautifulSoup(boxscore_page)
        data = data + getPitchingStats(boxscore_page,oppList[i])
    return data

In [85]:
data = getAllTeamBattingStats(team_schedule_table)

In [86]:
df = pd.DataFrame(data)

In [87]:
print df

   BB  ER   H    IP    R  SO                     name  \
0   0   0  23   6     93   0  Drew Hutchison, W (1-0)   
1   0   0   7   1.2   25   0               Aaron Loup   
2   0   0   3   1.1   12   0            Miguel Castro   
3   0   0  33   9    130   0              Team Totals   
4   0   0  24   6.1   94   0              R.A. Dickey   
5   0   0   2   0.2    4   0     Miguel Castro, H (1)   
6   0   0   3   0      8   0      Aaron Loup, L (0-1)   
7   0   0   4   0.1   16   0      Brett Cecil, BS (1)   
8   0   0   2   0.2   11   0            Roberto Osuna   
9   0   0  35   8    133   0              Team Totals   

                                          team_faced  
0  <td align="left"><a href="/teams/NYY/2015.shtm...  
1  <td align="left"><a href="/teams/NYY/2015.shtm...  
2  <td align="left"><a href="/teams/NYY/2015.shtm...  
3  <td align="left"><a href="/teams/NYY/2015.shtm...  
4  <td align="left"><a href="/teams/NYY/2015.shtm...  
5  <td align="left"><a href="/teams/NYY/20

In [16]:
df.to_excel('test.xlsx', sheet_name='sheet1', index=False)

ImportError: No module named openpyxl